In [1]:
import csv
from itertools import product
import numpy as np
import os
import pandas as pd
import requests
import time
from urllib2 import urlopen
import urllib2

In [2]:
#open files
df_raw_herb_hmdb_pair = pd.read_csv('/home/share/aschoi/data/COCONUT_Herbal/5_Similarity/7_sorted_score_vector_except_1(descending)_top_10_percent_no_score.csv', header=None)
df_hmdb = pd.read_table('/home/share/aschoi/data/COCONUT_Herbal/4_HMDB/HMDB_KEGG.tsv')
df_herb = pd.read_table('/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/2-5_herb_pubchem.tsv')
df_kegg_rpair = pd.read_table('/home/share/aschoi/data/COCONUT_Herbal/1_KEGG_rpair/KEGG_Rpair.txt')
df_orthology = pd.read_table('/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/3_all_predicted_orthology_wo_duplicate.tsv')

In [6]:
df_raw_herb_hmdb_pair.head(1)

,0,1
0,80303,HMDB37761


In [9]:
# extract elements of top percentage.
def count_percentage(percentage):
    print"Count the number of top %.2f %% similar compounds.." % percentage
    WHOLE_LENGTH = 646647053
    prepared_idx = 0
    percentage_idx = int(WHOLE_LENGTH * percentage / 100)
    
    if percentage <= 0.1 and percentage > 0 :
        df_top_herb_hmdb_pair = df_raw_herb_hmdb_pair[:percentage_idx]
        connect = 0
        print "There are %d similar compounds." % (prepared_idx + percentage_idx)
    elif percentage == 0 or percentage > 10 :
        #if percentage > 10 :
            #print("Input percentage is too big. Count just Top 10%.")
        #else percentage == 0 :
            #print("Input percentage is not valid.")
        df_top_herb_hmdb_pair = pd.DataFrame()
        connect = 1
    else :
        prepared_idx = int(WHOLE_LENGTH * 0.1 / 100)
        df_top_herb_hmdb_pair = df_raw_herb_hmdb_pair[prepared_idx+1:percentage_idx]
        connect = 2
        print "There are %d similar compounds." % (prepared_idx + percentage_idx)
    return connect, df_top_herb_hmdb_pair

In [10]:
#top herb id, top hmdb id
#def extract_top_id(df_top_herb_hmdb_pair):
#    top_herb = list()
#    top_hmdb = list()
#    for i in top_herb_hmdb_pair:
#        top_herb.append(int(i[0]))
#        top_hmdb.append(i[1].rstrip('\r\n'))
#    print("extract_top_ids")
#    return top_herb, top_hmdb

In [11]:
def find_keggid_of_hmdb(df_hmdb, top_hmdb):
    # The flag is used to find the KEGG ID of top_similar_hmdb.
    flag_find_keggid_of_top_hmdb = df_hmdb['Accession'].isin(top_hmdb)

    # This dataframe represents the pair of top hmdb id and its KEGG ID.
    # This information is used when find the substrate of KEGG rpair which correspond to the top_hmdb.
    df_top_hmdb_keggid_pair = df_hmdb[flag_find_keggid_of_top_hmdb][['Accession', 'KEGG_ID']]
    print("Find kegg id of hmdb")
    return df_top_hmdb_keggid_pair

In [12]:
def find_pubchemid_of_herb(df_herb, top_herb):
    # The flag is used to find the PubChem ID of top_herb.
    flag_find_pubchemid_of_top_herb = df_herb['compID'].isin(top_herb)

    # This dataframe represents the pair of top herb id and its PubChem ID.
    # Using this information, we can predict the orphan herbs' ontology.
    # One pubchem id of herb becomes the input substrate of KEGG E-zyme2 tool.
    df_top_herb_pubchemid_pair = df_herb[flag_find_pubchemid_of_top_herb][['compID', 'pubchemID']]
    print("find pubchemid of herb")
    return df_top_herb_pubchemid_pair

In [13]:
# Find the rpairs whose substrate is same to the top_hmdb.
# The pair is represented by only KEGG ID. (NOT hmdb_id.)
def find_kegg_rpair_of_hmdb_sub(df_kegg_rpair, df_top_hmdb_keggid_pair):
    flag_find_rpair_of_top_hmdb_sub = df_kegg_rpair['Compound1'].isin(df_top_hmdb_keggid_pair['KEGG_ID']) 

    # This dataframe consist of KEGG ID.
    df_rpair_of_top_hmdb_sub_k = df_kegg_rpair[flag_find_rpair_of_top_hmdb_sub]
    print("find kegg rpair of hmdb sub")
    return df_rpair_of_top_hmdb_sub_k

In [14]:
# Find the part of top_hmdb. 
# It have KEGG ID which corresponds to the substrate of KEGG rpair.
# Why need this process? 
# Because we don't have the pair of the kegg_id version substrate(although it corresponds to the top_hmdb!) and top_hmdb id.

def find_keggid_of_top_hmdb2(df_top_hmdb_keggid_pair, df_rpair_of_top_hmdb_sub_k):

    flag_find_hmdbid_of_top_hmdb_sub = df_top_hmdb_keggid_pair['KEGG_ID'].isin(df_rpair_of_top_hmdb_sub_k['Compound1'])
    df_top_hmdb_of_sub = df_top_hmdb_keggid_pair[flag_find_hmdbid_of_top_hmdb_sub]
    print("find keggid of top hmdb2")
    return df_top_hmdb_of_sub

In [15]:
# Find the pair of top_hmdb and top_herb.
# The top_hmdb is the substrate of the KEGG rpair.
# The top_herb is the similar herbal compounds of the top_hmdb.
def find_candidate_substrate(df_top_herb_hmdb_pair, df_top_hmdb_of_sub):
    flag_find_sim_herb_of_hmdb_sub = df_top_herb_hmdb_pair['Accession'].isin(df_top_hmdb_of_sub['Accession'])
    df_sim_herb_of_hmdb_sub = df_top_herb_hmdb_pair[flag_find_sim_herb_of_hmdb_sub]
    df_candidate_sub = pd.merge(df_top_hmdb_of_sub, df_sim_herb_of_hmdb_sub, how = 'inner', on = ['Accession', 'Accession'])
    print("find candidate substrate")
    return df_candidate_sub

In [16]:
# Make the pair of herb(used to be substrate) and product.
# If the pair will exist in the prepared orthology table, just show the predicted_result.
# Unless the pair will not exist in the prepared table, we have to crawl the predicted reusult from KEGG E-zyme2 webpage.
def query_list(df_candidate_sub, df_rpair_of_top_hmdb_sub_k):
    s_candidate_keggid = df_candidate_sub['KEGG_ID'].drop_duplicates()
    df_query = pd.DataFrame()
    for i in s_candidate_keggid:
        df_candidate_sub_part = df_candidate_sub[df_candidate_sub.KEGG_ID == i]
        df_candidate_pro_part = df_rpair_of_top_hmdb_sub_k[df_rpair_of_top_hmdb_sub_k.Compound1 == i]
        s_candidate_sub_part = df_candidate_sub_part['compID']
        s_candidate_pro_part = df_candidate_pro_part['Compound2']
        temp_df = pd.DataFrame(list(product(s_candidate_sub_part, s_candidate_pro_part)), columns = ['substrate', 'product'])
        df_query = df_query.append(temp_df)
    df_query.index = range(0, len(df_query))
    print("make query list")
    return df_query

In [17]:
time.strftime('%a %H:%M:%S')

'Wed 21:49:29'

In [18]:
def match_query_result(df_query, df_orthology):
    print("start match query result : " + time.strftime('%a %H:%M:%S'))
    predict_result = pd.DataFrame()
    for index, row in df_query.iterrows():
        query = row.tolist()
        result = df_orthology.query('@query[0] == substrate and @query[1] == product')
        predict_result = predict_result.append(result)
        #predict_result.index = range(0, len(predict_result))
    print("end match query result" + time.strftime('%a %H:%M:%S'))
    return predict_result

In [19]:
#def make_query_list(top_herb, top_hmdb):
def make_query_list(df_top_herb_hmdb_pair):
    
    # Dataframe of top_herb_hmdb_pair
    #df_top_herb_hmdb_pair = pd.DataFrame({'Accession':top_hmdb, 'compID' :top_herb})
    df_top_herb_hmdb_pair.rename(columns={0:'compID', 1:'Accession'})
    
    # Find the KEGG ID of top_hmdb.
    df_top_hmdb_keggid_pair = find_keggid_of_hmdb(df_hmdb, top_hmdb)
    
    # Find the PubChem ID of top_herb.
    df_top_herb_pubchemid_pair = find_pubchemid_of_herb(df_herb, top_herb)
    
    # Find the rpairs whose substrate is same to the top_hmdb.
    df_rpair_of_top_hmdb_sub_k = find_kegg_rpair_of_hmdb_sub(df_kegg_rpair, df_top_hmdb_keggid_pair)
    
    # Find the part of top_hmdb. 
    df_top_hmdb_of_sub = find_keggid_of_top_hmdb2(df_top_hmdb_keggid_pair, df_rpair_of_top_hmdb_sub_k)
    
    # Find the pair of top_hmdb and top_herb.
    df_candidate_sub = find_candidate_substrate(df_top_herb_hmdb_pair, df_top_hmdb_of_sub)
    
    # Make the pair of herb(used to be substrate) and product.
    df_query = query_list(df_candidate_sub, df_rpair_of_top_hmdb_sub_k)
    
    return df_query

In [20]:
def web_ezyme2(df_query):
    
    print "the number of crawling pair : %d " % len(df_query)
    predicted_result2 = pd.DataFrame()
    cnt = 0 
    url_base = 'http://rest.genome.jp/ezyme/'
    
    for index, row in df_query[cnt:].iterrows():
        sub = int(row['substrate'])
        pro = row['product']
        url = url_base + 'CID'+str(sub) +'/' + pro
        
        try:
            f = urlopen(url)
            all_data = f.readlines()
            data = all_data[1].split('\t')
            predicted_orthology = data[1]
            predicted_score = data[0]
        
            temp_df = pd.DataFrame([[sub, pro, predicted_orthology, predicted_score]], columns = ['substrate', 'product', 'predicted_orthology', 'predicted_score'])
            predicted_result2 = predicted_result2.append(temp_df)
        
        except urllib2.HTTPError, e:
            cnt = cnt + 1
            continue
    
    predicted_result2.index = range(0, len(predicted_result2))
    return predicted_result2

In [21]:
###### Main ###########

# aceeptuser input
percentage = input('Enter the percentage number (0,10] : ')

connect, df_top_herb_hmdb_pair = count_percentage(percentage)
if connect == 1:
    print("Input is not valid. Stop the program.")
else:
    
    #top_herb, top_hmdb =extract_top_id(df_top_herb_hmdb_pair)
    #df_query = make_query_list(top_herb, top_hmdb)
    df_query = make_query_list(df_top_herb_hmdb_pair)
    if connect == 2:
        print("need to connect the web. time consuming.. : " + time.strftime('%a %H:%M:%S'))
        predicted_result = web_ezyme2(df_query)
        print("end of web crawling : " + time.strftime('%a %H:%M:%S'))
        predicted_result = predicted_result.append(df_orthology) # prepared result (percentage under 0.1%)
        print("There are %d predicted orthology results.") % len(predicted_result)
    else:
        if percentage == 0.1:
            predicted_result = df_orthology
        else:
            predicted_result = match_query_result(df_query, df_orthology)
    print("There are %d predicted orthology results.") % len(predicted_result)

Enter the percentage number (0,10] : 0.001
Count the number of top 0.00 % similar compounds..
There are 6466 similar compounds.


NameError: global name 'top_hmdb' is not defined

In [1]:
predicted_result

NameError: name 'predicted_result' is not defined

In [81]:
WHOLE_LENGTH = 646647053
percentage = input('Enter the percentage number (ex. 0.1) : ')
percentage_idx = int(WHOLE_LENGTH * percentage / 100)
percentage_idx

Enter the percentage number (ex. 0.1) : 0.10001


646711

In [ ]:
0.1퍼센트 : 646647개. --> 0.10005 : 646970개 --> 0.10001 : 646711개

In [ ]:
#open file
#f = open('/home/share/aschoi/data/COCONUT_Herbal/5_Similarity/7_sorted_score_vector_except_1(descending)_top_10_percent_no_score.csv', 'r')
#raw_herb_hmdb_pair = []
#print time.strftime('%a %H:%M:%S')
#while True:
#    line = f.readline()
#    if not line: break
#    raw_herb_hmdb_pair.append(line.split(','))
#print time.strftime('%a %H:%M:%S')
#f.close()

In [ ]:
#open file
#f = open('/home/share/aschoi/data/COCONUT_Herbal/5_Similarity/7_sorted_score_vector_except_1(descending)_top_10_percent_no_score.csv', 'r')
#raw_herb_hmdb_pair = []
#print time.strftime('%a %H:%M:%S')
#while True:
#    line = f.readline()
#    if not line: break
#    raw_herb_hmdb_pair.append(line.split(','))
#print time.strftime('%a %H:%M:%S')
#f.close()

In [1]:
import csv
from itertools import product
import numpy as np
import os
import pandas as pd
import requests
import time
from urllib2 import urlopen
import urllib2

In [2]:
#open files
df_raw_herb_hmdb_pair = pd.read_csv('/home/share/aschoi/data/COCONUT_Herbal/5_Similarity/7_sorted_score_vector_except_1(descending)_top_10_percent_no_score.csv', header=None)
df_hmdb = pd.read_table('/home/share/aschoi/data/COCONUT_Herbal/4_HMDB/HMDB_KEGG.tsv')
df_herb = pd.read_table('/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/2-5_herb_pubchem.tsv')
df_kegg_rpair = pd.read_table('/home/share/aschoi/data/COCONUT_Herbal/1_KEGG_rpair/KEGG_Rpair.txt')
df_orthology = pd.read_table('/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/5_predicted_orthology.tsv')

In [3]:
# extract elements of top percentage.
#------------------------------------------------------------------------------
WHOLE_LENGTH = 646647053
percentage = input('Enter the percentage number (0,10] : ')
percentage_idx = int(WHOLE_LENGTH * percentage / 100)
df_top_herb_hmdb_pair = df_raw_herb_hmdb_pair[:percentage_idx]
df_top_herb_hmdb_pair = df_top_herb_hmdb_pair.rename(columns={0:'compID', 1:'Accession'})

Enter the percentage number (0,10] : 0.01


In [4]:
print len(df_top_herb_hmdb_pair)
df_top_herb_hmdb_pair.head()

64664


,compID,Accession
0,80303,HMDB37761
1,69881,HMDB02174
2,69881,HMDB02308
3,69881,HMDB03458
4,69881,HMDB06316


In [5]:
#top herb id, top hmdb id
#------------------------------------------------------------------------------
s_top_herb = df_top_herb_hmdb_pair['compID']
s_top_hmdb = df_top_herb_hmdb_pair['Accession']
print("    1. Extract the ids of top herbal and top HMDB compounds.")
#for i in top_herb_hmdb_pair:
#    top_herb.append(int(i[0]))
#    top_hmdb.append(i[1].rstrip('\r\n'))

    1. Extract the ids of top herbal and top HMDB compounds.


In [7]:
print len(s_top_herb)
print len(s_top_hmdb)

64664
64664


In [6]:
# Dataframe of top_herb_hmdb_pair
#df_top_herb_hmdb_pair = pd.DataFrame({'Accession':top_hmdb, 'compID' :top_herb})

In [8]:
# Find the KEGG ID of top_hmdb.

# The flag is used to find the KEGG ID of top_similar_hmdb.
flag_find_keggid_of_top_hmdb = df_hmdb['Accession'].isin(s_top_hmdb)

# This dataframe represents the pair of top hmdb id and its KEGG ID.
# This information is used when find the substrate of KEGG rpair which correspond to the top_hmdb.
df_top_hmdb_keggid_pair = df_hmdb[flag_find_keggid_of_top_hmdb][['Accession', 'KEGG_ID']]

In [9]:
print len(df_top_hmdb_keggid_pair)
df_top_hmdb_keggid_pair.head()

391


,Accession,KEGG_ID
6,HMDB00016,C03205
15,HMDB00032,C01164
24,HMDB00048,C05402
25,HMDB00050,C00212
28,HMDB00053,C00280


In [10]:
# Find the PubChem ID of top_herb.

# The flag is used to find the PubChem ID of top_herb.
flag_find_pubchemid_of_top_herb = df_herb['compID'].isin(s_top_herb)

# This dataframe represents the pair of top herb id and its PubChem ID.
# Using this information, we can predict the orphan herbs' ontology.
# One pubchem id of herb becomes the input substrate of KEGG E-zyme2 tool.
df_top_herb_pubchemid_pair = df_herb[flag_find_pubchemid_of_top_herb][['compID', 'pubchemID']]

In [11]:
print len(df_top_herb_pubchemid_pair)
df_top_herb_pubchemid_pair.head()

2854


,compID,pubchemID
9,33179,88810
24,62074,6325611
28,54483,11432549
34,62081,5319346
39,76855,5319337


In [12]:
# Find the rpairs whose substrate is same to the top_hmdb.
# The pair is represented by only KEGG ID. (NOT hmdb_id.)

flag_find_rpair_of_top_hmdb_sub = df_kegg_rpair['Compound1'].isin(df_top_hmdb_keggid_pair['KEGG_ID']) 

# This dataframe consist of KEGG ID.
df_rpair_of_top_hmdb_sub_k = df_kegg_rpair[flag_find_rpair_of_top_hmdb_sub]

In [13]:
print len(df_rpair_of_top_hmdb_sub_k)
df_rpair_of_top_hmdb_sub_k.head()

724


,Compound1,Compound2
1,C00003,C00004
47,C00008,C00020
51,C00031,C00092
64,C00089,C00095
70,C00003,C00153


In [14]:
# Find the part of top_hmdb. 
# It have KEGG ID which corresponds to the substrate of KEGG rpair.
# Why need this process? 
# Because we don't have the pair of the kegg_id version substrate(although it corresponds to the top_hmdb!) and top_hmdb id.

flag_find_hmdbid_of_top_hmdb_sub = df_top_hmdb_keggid_pair['KEGG_ID'].isin(df_rpair_of_top_hmdb_sub_k['Compound1'])
df_top_hmdb_of_sub = df_top_hmdb_keggid_pair[flag_find_hmdbid_of_top_hmdb_sub]

In [15]:
print len(df_top_hmdb_of_sub)
df_top_hmdb_of_sub.head()

173


,Accession,KEGG_ID
6,HMDB00016,C03205
15,HMDB00032,C01164
24,HMDB00048,C05402
25,HMDB00050,C00212
28,HMDB00053,C00280


In [16]:
# Find the pair of top_hmdb and top_herb.
# The top_hmdb is the substrate of the KEGG rpair.
# The top_herb is the similar herbal compounds of the top_hmdb.

flag_find_sim_herb_of_hmdb_sub = df_top_herb_hmdb_pair['Accession'].isin(df_top_hmdb_of_sub['Accession'])
df_sim_herb_of_hmdb_sub = df_top_herb_hmdb_pair[flag_find_sim_herb_of_hmdb_sub]

# Find the pubchem id of the herbs which is similar to the hmdb compounds. These hmdb compounds are the substrate of rpair. 
flag_find_pubchemid_of_sim_herb =df_top_herb_pubchemid_pair['compID'].isin(df_sim_herb_of_hmdb_sub['compID'])
df_sim_herb_pubchem_id_pair = df_top_herb_pubchemid_pair[flag_find_pubchemid_of_sim_herb]

df_candidate_sub_herb = pd.merge(df_top_hmdb_of_sub, df_sim_herb_of_hmdb_sub, how = 'inner', on = ['Accession', 'Accession'])
df_candidate_sub_herb_pubchem = pd.merge(df_candidate_sub_herb, df_sim_herb_pubchem_id_pair, how='inner', on=['compID', 'compID'])


In [17]:
print len(df_candidate_sub_herb_pubchem)
df_candidate_sub_herb_pubchem.head()

685


,Accession,KEGG_ID,compID,pubchemID
0,HMDB00016,C03205,76563,5318189
1,HMDB00234,C00535,76563,5318189
2,HMDB00374,C01176,76563,5318189
3,HMDB00016,C03205,76033,5318270
4,HMDB00234,C00535,76033,5318270


In [22]:
# Make the pair of herb(used to be substrate) and product.
# If the pair will exist in the prepared orthology table, just show the predicted_result.
# Unless the pair will not exist in the prepared table, we have to crawl the predicted reusult from KEGG E-zyme2 webpage.

from itertools import product
s_candidate_keggid = df_candidate_sub_herb_pubchem['KEGG_ID'].drop_duplicates()
df_query = pd.DataFrame()
for i in s_candidate_keggid:
    df_candidate_sub_part = df_candidate_sub_herb_pubchem[df_candidate_sub_herb_pubchem.KEGG_ID == i]
    df_candidate_pro_part = df_rpair_of_top_hmdb_sub_k[df_rpair_of_top_hmdb_sub_k.Compound1 == i]
    s_candidate_sub_part = df_candidate_sub_part['pubchemID']
    s_candidate_pro_part = df_candidate_pro_part['Compound2']
    temp_df = pd.DataFrame(list(product(s_candidate_sub_part, s_candidate_pro_part)), columns = ['substrate', 'product'])
    df_query = df_query.append(temp_df)
df_query = df_query.drop_duplicates()
df_query.index = range(0, len(df_query))

In [24]:
print "       inner query_list : len(df_candidate_sub_herb_pubchem, df_rpair_of_top_hmdb_sub_k) %d, %d" % (len(df_candidate_sub_herb_pubchem), len(df_rpair_of_top_hmdb_sub_k))

       inner query_list : len(df_candidate_sub_herb_pubchem, df_rpair_of_top_hmdb_sub_k) 685, 724


In [23]:
len(s_candidate_keggid)

154

In [21]:
print len(df_query)
df_query.head()

1830


,substrate,product
0,5318189,C05485
1,5318189,C04108
2,5318189,C18040
3,5318270,C05485
4,5318270,C04108


In [22]:
#다시 시작시 파일명 바꾸기
with open("/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/6_1_predicted_ontology.tsv", "w") as resultFile:
    features = ['substrate', 'product', 'predicted_orthology', 'predicted_score']
    resultTsv = csv.DictWriter(resultFile, features, dialect=csv.excel_tab)
    resultTsv.writeheader()
    
    #predicted_result = pd.DataFrame()
    cnt = 0
    url_base = 'http://rest.genome.jp/ezyme/'
    print '%d : '% cnt + time.strftime('%a %H:%M:%S')
    for index, row in df_query[cnt:].iterrows():
        sub = int(row['substrate'])
        pro = row['product']
        url = url_base + 'CID'+str(sub) +'/' + pro
        
        try:
            f = urlopen(url)
            all_data = f.readlines()
            data = all_data[1].split('\t')
            predicted_orthology = data[1]
            predicted_score = data[0]
            resultTsv.writerow({'substrate':sub, 'product':pro, 'predicted_orthology' : predicted_orthology, 'predicted_score' : predicted_score})     
            #temp_df = pd.DataFrame([[sub, pro, predicted_orthology, predicted_score]], columns = ['substrate', 'product', 'predicted_orthology', 'predicted_score'])
            #predicted_result2 = predicted_result2.append(temp_df)
            cnt = cnt + 1
            
        except urllib2.HTTPError, e:
            if e.code == 404:
                print 'HTTPError 404 not found %d : ' % cnt + time.strftime('%a %H:%M:%S')
                print url
                cnt = cnt + 1
                continue
            else:
                print e.code
                print url
                print cnt
                raise
            
        #except socket.error, exc:
            #print 'socket error {} ...'.format(exc[0])
            #print '<time out error %d> : ' % cnt + time.strftime('%a %H:%M:%S')
            #print url
            #f = urlopen(url)
            #all_data = f.readlines()
            #cnt = cnt + 1
            #data = all_data[1].split('\t')
            #predicted_orthology = data[1]
            #predicted_score = data[0]
            #resultTsv.writerow({'substrate':sub, 'product':pro, 'predicted_orthology' : predicted_orthology, 'predicted_score' : predicted_score})
            #continue
        if cnt % 1000 == 0 :
            print '%d : '% cnt + time.strftime('%a %H:%M:%S')
print time.strftime('%a %H:%M:%S')

0 : Fri 15:35:33
1000 : Fri 17:20:19
HTTPError 1607 : Fri 18:34:45
http://rest.genome.jp/ezyme/CID16758819/C02140
2000 : Fri 19:23:31


NameError: name 'socket' is not defined

In [ ]:
df4 = pd.read_table("/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/5_1_predicted_ontology.tsv")
df5 = pd.read_table("/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/5_2_predicted_ontology.tsv")
frames2=[df4, df5]
df6 = pd.concat(frames2)
print len(df6.drop_duplicates())
df7 = df6.drop_duplicates()

In [51]:
# 5_predicted_orthology를 돌릴때 HTTPerror이긴 한데 404가 아닌애들.
reurl = ['http://rest.genome.jp/ezyme/CID104745/C17238',
'http://rest.genome.jp/ezyme/CID104745/C20258',
'http://rest.genome.jp/ezyme/CID5281127/C14825',
'http://rest.genome.jp/ezyme/CID5281127/C14826',
'http://rest.genome.jp/ezyme/CID5281127/C14827',
'http://rest.genome.jp/ezyme/CID5281127/C14831',
'http://rest.genome.jp/ezyme/CID5281127/C14831',
'http://rest.genome.jp/ezyme/CID5281127/C14831',
'http://rest.genome.jp/ezyme/CID5282761/C16321',
'http://rest.genome.jp/ezyme/CID5282761/C16162',
'http://rest.genome.jp/ezyme/CID5282761/C19936',
'http://rest.genome.jp/ezyme/CID5282761/C20704']

In [70]:
#다시함.
for i in reurl:
    f2 = urlopen(i)
    all_data2 = f2.readlines()
    data2 = all_data2[1].split('\t')
    predicted_orthology2 = data2[1]
    predicted_score2 = data2[0]
    sub2 = i[-13:-7]
    pro2 = i[-6:]
    
    temp_df = pd.DataFrame([[sub2, pro2, predicted_orthology2, predicted_score2]], columns = ['substrate', 'product', 'predicted_orthology', 'predicted_score'])
    df7 = df7.append(temp_df)
    
    

In [74]:
print len(reurl)
print len(df7)
print len(df7) - len(reurl)

12
26205
26193


In [76]:
df8=df7.drop_duplicates()

In [77]:
df8.index = range(0, len(df8))

In [80]:
print len(df8)
df8.head()

26204


,substrate,product,predicted_orthology,predicted_score
0,3483736,C05665,K00276,0.381
1,3483736,C03375,K01844,0.053
2,3483736,C03283,K01580,0.736
3,3483736,C15853,K00632,0.106
4,3483736,C01029,K01844,0.044


In [79]:
df8.to_csv("/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/5_predicted_orthology.tsv", sep='\t', index=None)

In [48]:
#url2 = 'http://rest.genome.jp/ezyme/CID16758819/C02140' #HTTPerror 404 url
#url2 = 'http://rest.genome.jp/ezyme/CID104745/C17238'
#url2 ='http://rest.genome.jp/ezyme/CID89488/C05498'
try:
    f2 = urlopen(url2)
    all_data2 = f2.readlines()
    data2 = all_data2[1].split('\t')
    predicted_orthology2 = data2[1]
    predicted_score2 = data2[0]
except urllib2.HTTPError as err:
    if err.code == 404:
        raise


In [ ]:
if connect:
    predicted_result = web_ezyme2(df_query)
else:
    predict_result = pd.DataFrame()
    for index, row in df_query.iterrows():
        query = row.tolist()
        result = df_orthology.query('@query[0] == substrate and @query[1] == product')
        predict_result = predict_result.append(result)
        predict_result

In [221]:
def web_ezyme2(df_query):
      
    predicted_result2 = pd.DataFrame()
    cnt = 0 
    url_base = 'http://rest.genome.jp/ezyme/'
    
    for index, row in df_query[cnt:].iterrows():
        sub = int(row['substrate'])
        pro = row['product']
        url = url_base + 'CID'+str(sub) +'/' + pro
        
        try:
            f = urlopen(url)
            all_data = f.readlines()
            data = all_data[1].split('\t')
            predicted_orthology = data[1]
            predicted_score = data[0]
        
            temp_df = pd.DataFrame([[sub, pro, predicted_orthology, predicted_score]], columns = ['substrate', 'product', 'predicted_orthology', 'predicted_score'])
            predicted_result2 = predicted_result2.append(temp_df)
        
        except urllib2.HTTPError, e:
            cnt = cnt + 1
            continue
    
    predicted_result2.index = range(0, len(predicted_result2))
    return predicted_result2

In [193]:
predict_result

,substrate,product,predicted_orthology,predicted_score
31229,34964,C10175,K15506,0.137
31230,61907,C10175,K01730,0.092
31231,77622,C10175,K08681,0.013
31232,88970,C10175,K15506,0.131
31233,90406,C10175,K02611,0.085
31234,88968,C10175,K15506,0.108
31235,77625,C10175,K15506,0.125
31236,77626,C10175,K03781,0.201
31237,65001,C10175,K15506,0.123
40849,311,C01735,K15242,0.005
